# 第6回スキルトランスファー(眞田担当分)

## ハンズオン

### 週次のワランティ推移

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite:///../data/interim/suzuki_skilltransfer.sqlite3")

In [3]:
import pandas as pd

In [4]:
pd.set_option("display.max_columns", 1000)

In [5]:
pd.read_sql("select * from warranty limit 1", engine)

,W_KEY,W_BASE_CD,W_PROD_BASE_CD,W_FIELD_CLAIM_DIV,W_PROD_DIV,W_DMEX_DIV,W_PROCESS_MONTH,W_DIST_CD,W_DIST_COUNTRY_CD,W_DIST_ISSUE_NO,W_SHOP_DIV,W_SHOP_CD,W_DIV1,W_DIV2,W_DIV3,W_FIXED_DIV,W_MINUS_DIV,W_SRC_DIV,W_SRC_KEY,W_VIN,W_FCOK,W_PROD_MODEL_CD,W_SALES_DATE,W_FAILURE_DATE,W_MILEAGE_DIV,W_MILEAGE,W_CAUSAL_PART_NO,W_TROUBLE_CC,W_TROUBLE_CP,W_TROUBLE_DF,W_TROUBLE_MP,W_DTC_1,W_DTC_2,W_OPERATION_CD,W_PARTS_AMT,W_LABOR_HOUR,W_LABOR_AMT,W_SUBLET_HOUR,W_SUBLET_AMT,W_SHIPPING_AMT,W_TOTAL_AMT,W_CFC_COMPANY_CD,W_CFC_RATE,W_CFC_AMT,W_FC_TO_FAILURE,W_SALES_TO_FAILURE,W_SB_NO,W_CRT_TIME_STAMP
0,WRAEJP201702A000001,100100,100100,2,A,1,201702,2505A,AE,00A10709,1,None,None,2,0,1,1,1,2017022505A00A10709,JS1VY54A682102346,20080201,XD021,20080723,20170122,K,10497,32800-10820-RX0,None,99,QR,None,None,None,SF9999,2266.0,0.4,1200.0,0.0,0.0,679.0,4145.0,P58X,0,0.0,107,102,None,17-08-02 18:34:49.631885000


In [6]:
with pd.option_context("display.max_columns", 1000):
    display(
        pd.read_sql("select * from warranty limit 1"
                    , engine)
    )

,W_KEY,W_BASE_CD,W_PROD_BASE_CD,W_FIELD_CLAIM_DIV,W_PROD_DIV,W_DMEX_DIV,W_PROCESS_MONTH,W_DIST_CD,W_DIST_COUNTRY_CD,W_DIST_ISSUE_NO,W_SHOP_DIV,W_SHOP_CD,W_DIV1,W_DIV2,W_DIV3,W_FIXED_DIV,W_MINUS_DIV,W_SRC_DIV,W_SRC_KEY,W_VIN,W_FCOK,W_PROD_MODEL_CD,W_SALES_DATE,W_FAILURE_DATE,W_MILEAGE_DIV,W_MILEAGE,W_CAUSAL_PART_NO,W_TROUBLE_CC,W_TROUBLE_CP,W_TROUBLE_DF,W_TROUBLE_MP,W_DTC_1,W_DTC_2,W_OPERATION_CD,W_PARTS_AMT,W_LABOR_HOUR,W_LABOR_AMT,W_SUBLET_HOUR,W_SUBLET_AMT,W_SHIPPING_AMT,W_TOTAL_AMT,W_CFC_COMPANY_CD,W_CFC_RATE,W_CFC_AMT,W_FC_TO_FAILURE,W_SALES_TO_FAILURE,W_SB_NO,W_CRT_TIME_STAMP
0,WRAEJP201702A000001,100100,100100,2,A,1,201702,2505A,AE,00A10709,1,None,None,2,0,1,1,1,2017022505A00A10709,JS1VY54A682102346,20080201,XD021,20080723,20170122,K,10497,32800-10820-RX0,None,99,QR,None,None,None,SF9999,2266.0,0.4,1200.0,0.0,0.0,679.0,4145.0,P58X,0,0.0,107,102,None,17-08-02 18:34:49.631885000


In [8]:
# 故障日, パーツ種類(上9桁), モデル品番(上3桁), 件数
# w_field_claim_div = 1, 
# w_prod_base_cd = 100100, w_prod_div = "B", w_dmex_div = 0

In [9]:
df_warranty_daily = pd.read_sql(
"""
select W_FAILURE_DATE as date
       ,substr(W_CAUSAL_PART_NO, 1, 9) as part
       ,substr(W_PROD_MODEL_CD, 1, 3) as model
       ,count(*) as n
from warranty
where w_prod_base_cd = '100100'
and w_prod_div = 'B'
and w_dmex_div = '0'
and w_field_claim_div = '1'
group by part, model, date;
""", engine)

In [10]:
df_warranty_daily.head()

,date,part,model,n
0,20170602,01421-102,YD3,1
1,20170907,01547-064,Y3J,1
2,20171028,01550-062,YV7,1
3,20170128,01550-065,YR6,1
4,20170328,01550-082,YMA,1


In [11]:
df_warranty_daily.shape

(159930, 4)

In [12]:
df_warranty_daily.date = \
    pd.to_datetime(df_warranty_daily.date)

datetime -> str(format)

In [13]:
df_warranty_daily.set_index("date", inplace=True)

In [14]:
# Month Start
# Month
df_warranty_daily.resample("MS")["n"].min()

date
2017-01-01    1
2017-02-01    1
2017-03-01    1
2017-04-01    1
2017-05-01    1
2017-06-01    1
2017-07-01    1
2017-08-01    1
2017-09-01    1
2017-10-01    1
2017-11-01    1
2017-12-01    1
Freq: MS, Name: n, dtype: int64

https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html

https://note.nkmk.me/python-pandas-time-series-freq/

In [16]:
df_warranty_daily.reset_index(inplace=True)

In [17]:
df_warranty_daily.head()

,date,part,model,n
0,2017-06-02,01421-102,YD3,1
1,2017-09-07,01547-064,Y3J,1
2,2017-10-28,01550-062,YV7,1
3,2017-01-28,01550-065,YR6,1
4,2017-03-28,01550-082,YMA,1


In [18]:
df_warranty_weekly = (df_warranty_daily
                      .groupby(["part",
                                pd.Grouper(key="date",
                                           freq="W")])["n"]
                      .sum().reset_index())

In [19]:
df_warranty_weekly.head()

,part,date,n
0,01421-102,2017-06-04,1
1,01547-064,2017-09-10,1
2,01550-062,2017-10-29,1
3,01550-065,2017-01-29,1
4,01550-082,2017-04-02,1


In [20]:
df_warranty_weekly.date = df_warranty_weekly.date \
    - pd.to_timedelta(7, unit="d")

In [21]:
df_warranty_weekly.head()

,part,date,n
0,01421-102,2017-05-28,1
1,01547-064,2017-09-03,1
2,01550-062,2017-10-22,1
3,01550-065,2017-01-22,1
4,01550-082,2017-03-26,1


### 経過月毎のワランティ推移

In [22]:
# select:
#   故障までの月(W_SALES_TO_FAILURE), 
#   パーツ種類(上9桁)(W_CAUSAL_PART_NO),
#   モデル品番(上3桁)(W_PROD_MODEL_CD),
#   件数
# 条件(where:)
# w_field_claim_div = 1, 
# w_prod_base_cd = 100100, w_prod_div = "B", w_dmex_div = 0

In [23]:
df_warranty_stf = pd.read_sql(
"""
select W_SALES_TO_FAILURE as stf
       ,substr(W_CAUSAL_PART_NO, 1, 9) as part
       ,substr(W_PROD_MODEL_CD, 1, 3) as model
       ,count(*) as n
from warranty
where w_base_cd = '100100'
and w_prod_base_cd = '100100'
and w_prod_div = 'B'
and w_dmex_div = '0'
and w_field_claim_div = '1'
group by part, model, stf;
""", engine) # 

In [24]:
# ハンガリーが混ざる / 両方 -> 100100

In [25]:
# パーツコードマスタを作成。どの主原因品番がどのモデルに含まれているのかの読み替え表を作成

df_part_model_master = \
    (df_warranty_daily[["part", "model"]]
                       .dropna()
                       .drop_duplicates())

# 少数のマスターの調整

In [26]:
tmp = df_warranty_daily.groupby(["part", "model"])["n"].sum()

In [27]:
tmp = tmp[tmp.values >= 5] # 5以上

In [28]:
tmp.reset_index().loc[:,["part", "model"]].head()

,part,model
0,01551-081,YV7
1,02162-061,YJA
2,02162-061,YW0
3,09103-083,YN1
4,09103-120,YH7


In [29]:
df_part_model_master.head()

,part,model
0,01421-102,YD3
1,01547-064,Y3J
2,01550-062,YV7
3,01550-065,YR6
4,01550-082,YMA


In [30]:
df_warranty_stf.head()

,stf,part,model,n
0,36,01421-102,YD3,1
1,10,01547-064,Y3J,1
2,33,01550-062,YV7,1
3,39,01550-065,YR6,1
4,31,01550-082,YMA,1


## 宿題

- 6.2.1_AkihiroSanada_TableMaking_Check_revised.ipynbを実行する。途中の式を眺めて、理解する。
    - 質問があれば、Teamsで何でも聞いて下さい

### 追加の宿題

- 経過月毎のワランティ推移について、6.2.1フーリエ変換を行う。結果について考察する。

- 時系列分析が使えそうなユースケースについて、1つ考える。そのユースケースにおいて、どのようなテーブルが必要か考える(縦軸を構成するインデックス列、横軸を構成する項目)